# 🚀 Insurance LLaMA Interactive Demo

This notebook provides an interactive demo of the fine-tuned LLaMA model for insurance tasks:

## What this notebook does:
1. Load the fine-tuned insurance model
2. Provide interactive demos for each task type
3. Allow custom input testing
4. Compare with baseline model (optional)
5. Showcase real-world insurance use cases
6. Generate sample outputs for different scenarios

**⚠️ Important: Make sure you have a trained model from notebook 03**

## 1. Import Libraries and Setup

In [ ]:
import os
import json
import torch
import warnings
from pathlib import Path
from typing import Dict, List, Optional
from datetime import datetime

# Core ML libraries
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    GenerationConfig
)
from peft import PeftModel

# Interactive widgets for Colab/Jupyter
try:
    from ipywidgets import interact, widgets, Layout
    from IPython.display import display, HTML, clear_output
    WIDGETS_AVAILABLE = True
except ImportError:
    print("ipywidgets not available - using basic interface")
    WIDGETS_AVAILABLE = False

warnings.filterwarnings('ignore')

print("✅ Libraries imported successfully")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
print(f"Interactive widgets: {'Available' if WIDGETS_AVAILABLE else 'Not available'}")

## 2. Configuration and Model Loading

In [ ]:
# Model paths
BASE_MODEL_NAME = "meta-llama/Llama-2-7b-chat-hf"
LORA_MODEL_PATH = Path("outputs/final_model/lora_model")

# Generation configurations for different tasks
GENERATION_CONFIGS = {
    'creative': {
        'max_new_tokens': 512,
        'temperature': 0.8,
        'top_p': 0.9,
        'top_k': 50,
        'do_sample': True,
        'repetition_penalty': 1.1
    },
    'factual': {
        'max_new_tokens': 256,
        'temperature': 0.3,
        'top_p': 0.8,
        'top_k': 40,
        'do_sample': True,
        'repetition_penalty': 1.05
    },
    'precise': {
        'max_new_tokens': 128,
        'temperature': 0.1,
        'top_p': 0.7,
        'top_k': 20,
        'do_sample': True,
        'repetition_penalty': 1.02
    }
}

# Insurance task templates
TASK_TEMPLATES = {
    'CLAIM_CLASSIFICATION': {
        'instruction': 'Classify this insurance claim into the appropriate category (auto, health, life, property, etc.):',
        'config': 'precise'
    },
    'POLICY_SUMMARIZATION': {
        'instruction': 'Summarize the key points of this insurance policy in simple terms:',
        'config': 'factual'
    },
    'FAQ_GENERATION': {
        'instruction': 'Generate 3-5 frequently asked questions and answers based on this insurance information:',
        'config': 'creative'
    },
    'COMPLIANCE_CHECK': {
        'instruction': 'Identify the key compliance and regulatory requirements mentioned in this insurance document:',
        'config': 'factual'
    },
    'CONTRACT_QA': {
        'instruction': 'Answer this question based on the insurance contract information provided:',
        'config': 'precise'
    }
}

print(f"Configuration loaded:")
print(f"- Base model: {BASE_MODEL_NAME}")
print(f"- LoRA model path: {LORA_MODEL_PATH}")
print(f"- Generation configs: {list(GENERATION_CONFIGS.keys())}")
print(f"- Available tasks: {list(TASK_TEMPLATES.keys())}")

## 3. Load Fine-tuned Model

In [ ]:
class InsuranceLLaMAModel:
    """Wrapper class for the fine-tuned insurance LLaMA model"""
    
    def __init__(self, model_path: Path, base_model_name: str):
        self.model_path = model_path
        self.base_model_name = base_model_name
        self.model = None
        self.tokenizer = None
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    def load_model(self):
        """Load the fine-tuned model and tokenizer"""
        
        print(f"Loading fine-tuned model from {self.model_path}...")
        
        if not self.model_path.exists():
            raise FileNotFoundError(f"Model not found at {self.model_path}. Please run training notebook first.")
        
        try:
            # Load tokenizer
            self.tokenizer = AutoTokenizer.from_pretrained(self.model_path)
            
            # Load base model
            print(f"Loading base model: {self.base_model_name}")
            base_model = AutoModelForCausalLM.from_pretrained(
                self.base_model_name,
                torch_dtype=torch.float16,
                device_map="auto",
                trust_remote_code=True
            )
            
            # Load LoRA model
            print(f"Loading LoRA adapters...")
            self.model = PeftModel.from_pretrained(base_model, self.model_path)
            
            # Set pad token
            if self.tokenizer.pad_token is None:
                self.tokenizer.pad_token = self.tokenizer.eos_token
            
            print(f"✅ Model loaded successfully!")
            print(f"  Device: {next(self.model.parameters()).device}")
            print(f"  Vocab size: {len(self.tokenizer)}")
            
        except Exception as e:
            print(f"❌ Error loading model: {e}")
            raise
    
    def generate_response(self, prompt: str, generation_config: Dict) -> str:
        """Generate response from the model"""
        
        if self.model is None:
            raise RuntimeError("Model not loaded. Call load_model() first.")
        
        try:
            # Add pad_token_id to generation config
            gen_config = generation_config.copy()
            gen_config['pad_token_id'] = self.tokenizer.pad_token_id
            gen_config['eos_token_id'] = self.tokenizer.eos_token_id
            
            # Tokenize input
            inputs = self.tokenizer(prompt, return_tensors="pt", padding=True, truncation=True, max_length=2048)
            inputs = {k: v.to(self.model.device) for k, v in inputs.items()}
            
            # Generate response
            with torch.no_grad():
                outputs = self.model.generate(
                    **inputs,
                    **gen_config,
                    use_cache=True
                )
            
            # Decode response (remove input prompt)
            input_length = inputs['input_ids'].shape[1]
            generated_tokens = outputs[0][input_length:]
            response = self.tokenizer.decode(generated_tokens, skip_special_tokens=True)
            
            return response.strip()
            
        except Exception as e:
            print(f"⚠️ Error generating response: {e}")
            return "Error generating response. Please try again."
    
    def format_insurance_prompt(self, task_type: str, user_input: str, context: str = "") -> str:
        """Format input into proper instruction prompt for insurance tasks"""
        
        if task_type not in TASK_TEMPLATES:
            raise ValueError(f"Unknown task type: {task_type}")
        
        template = TASK_TEMPLATES[task_type]
        instruction = template['instruction']
        
        if task_type == 'CONTRACT_QA' and context:
            full_input = f"Context: {context}\n\nQuestion: {user_input}"
        else:
            full_input = user_input
        
        prompt = f"[INST] {instruction}\n\n{full_input} [/INST]"
        return prompt
    
    def process_insurance_task(self, task_type: str, user_input: str, context: str = "") -> Dict:
        """Process a complete insurance task"""
        
        # Format prompt
        prompt = self.format_insurance_prompt(task_type, user_input, context)
        
        # Get generation config
        config_name = TASK_TEMPLATES[task_type]['config']
        generation_config = GENERATION_CONFIGS[config_name]
        
        # Generate response
        response = self.generate_response(prompt, generation_config)
        
        return {
            'task_type': task_type,
            'user_input': user_input,
            'context': context,
            'prompt': prompt,
            'response': response,
            'config_used': config_name,
            'timestamp': datetime.now().isoformat()
        }

# Initialize and load the model
print("Initializing Insurance LLaMA Model...")
insurance_model = InsuranceLLaMAModel(LORA_MODEL_PATH, BASE_MODEL_NAME)

try:
    insurance_model.load_model()
    MODEL_LOADED = True
    print("\n🎉 Model ready for inference!")
except Exception as e:
    print(f"❌ Failed to load model: {e}")
    print("\nPlease ensure you have:")
    print("1. Completed training in notebook 03_finetuning_lora.ipynb")
    print("2. Model files saved in outputs/final_model/lora_model/")
    print("3. Sufficient GPU memory available")
    MODEL_LOADED = False

## 4. Sample Insurance Scenarios

In [ ]:
# Sample insurance data for demos
SAMPLE_SCENARIOS = {
    'CLAIM_CLASSIFICATION': [
        "Vehicle collision on I-95 involving two cars. Front-end damage to insured vehicle. No injuries reported. Police report filed. Towing required.",
        "Water damage to kitchen due to burst pipe. Hardwood floors damaged, cabinets need replacement. Plumber has been contacted.",
        "Annual wellness checkup at primary care physician. Routine blood work and physical examination. No issues found.",
        "House fire caused by electrical fault. Significant damage to roof and upper floor. Fire department response required."
    ],
    'POLICY_SUMMARIZATION': [
        """COMPREHENSIVE AUTO INSURANCE POLICY
        
Coverage Limits: Bodily Injury Liability $100,000 per person, $300,000 per accident. Property Damage Liability $50,000 per accident. Comprehensive and Collision coverage with $500 deductible each.

Premium: $1,200 annually, paid in monthly installments of $100. Late payment fee of $25 applies after 10-day grace period.

Exclusions: Racing, commercial use, intentional damage, normal wear and tear, mechanical breakdown not covered.

Additional Benefits: Roadside assistance, rental car coverage up to $30/day for 30 days, new car replacement if total loss occurs within first year.""",
        
        """HEALTH INSURANCE POLICY
        
Plan Type: PPO with nationwide network coverage. Annual deductible $2,000 individual, $4,000 family.

Coverage: 80% coinsurance after deductible for in-network providers, 60% for out-of-network. Preventive care covered 100%.

Prescription Benefits: $10 generic, $30 brand name, $60 specialty drugs. 90-day supply available by mail.

Maximum Out-of-Pocket: $8,000 individual, $16,000 family per year."""
    ],
    'FAQ_GENERATION': [
        """HOME INSURANCE BASICS
        
Homeowners insurance protects your home and personal belongings from covered perils like fire, theft, and storms. It also provides liability coverage if someone is injured on your property.

Standard policies cover the dwelling, other structures, personal property, and liability. Additional living expenses are covered if your home becomes uninhabitable.

Common exclusions include floods, earthquakes, and normal wear and tear. These may require separate policies or endorsements.""",
        
        """LIFE INSURANCE OVERVIEW
        
Life insurance provides financial protection for your beneficiaries when you die. Term life provides coverage for a specific period, while whole life provides permanent coverage with cash value.

Premiums depend on age, health, coverage amount, and policy type. Medical exams are typically required for larger policies.

Beneficiaries receive death benefits tax-free. Policies can also be used for estate planning and business protection."""
    ],
    'COMPLIANCE_CHECK': [
        """INSURANCE COMPANY OPERATIONS
        
All insurance operations must comply with state insurance regulations and maintain adequate reserves. Companies must file annual financial statements with state commissioners.

Consumer protection laws require clear policy language, fair claims handling, and prompt payment of valid claims. Unfair trade practices are prohibited.

Health insurance operations must comply with HIPAA privacy requirements and ACA regulations including essential health benefits and pre-existing condition protections.

All marketing materials must be approved by the state and cannot be misleading or deceptive."""
    ],
    'CONTRACT_QA': [
        {
            'context': """AUTO INSURANCE POLICY - SECTION 4: COLLISION COVERAGE
            
We will pay for direct and accidental loss to your covered auto caused by collision with another object or by upset of your covered auto. We will pay the lesser of: (1) actual cash value minus applicable deductible, or (2) amount necessary to repair or replace the property minus applicable deductible.
            
Your deductible is $500 per occurrence. Collision coverage applies only if you have purchased this coverage, as shown in the Declarations.""",
            'questions': [
                "What is my collision deductible?",
                "Does collision coverage apply to all accidents?",
                "How much will I receive if my car is totaled?"
            ]
        }
    ]
}

print("✅ Sample scenarios loaded")
print(f"Available demo scenarios:")
for task, scenarios in SAMPLE_SCENARIOS.items():
    if task == 'CONTRACT_QA':
        print(f"  {task}: {len(scenarios[0]['questions'])} Q&A examples")
    else:
        print(f"  {task}: {len(scenarios)} scenarios")

## 5. Demo Functions

In [ ]:
def run_sample_demo(task_type: str, sample_index: int = 0):
    """Run a sample demo for a specific task type"""
    
    if not MODEL_LOADED:
        print("❌ Model not loaded. Please load the model first.")
        return
    
    if task_type not in SAMPLE_SCENARIOS:
        print(f"❌ Unknown task type: {task_type}")
        return
    
    print(f"🎯 {task_type} Demo")
    print("=" * 60)
    
    scenarios = SAMPLE_SCENARIOS[task_type]
    
    if task_type == 'CONTRACT_QA':
        # Handle Q&A scenarios
        scenario = scenarios[0]
        context = scenario['context']
        questions = scenario['questions']
        
        print(f"Context: {context[:200]}...")
        print(f"\nAnswering {len(questions)} questions:\n")
        
        for i, question in enumerate(questions, 1):
            print(f"Q{i}: {question}")
            
            result = insurance_model.process_insurance_task(task_type, question, context)
            
            print(f"A{i}: {result['response']}")
            print("-" * 40)
    
    else:
        # Handle other task types
        if sample_index >= len(scenarios):
            sample_index = 0
        
        scenario = scenarios[sample_index]
        
        print(f"Input: {scenario[:300]}{'...' if len(scenario) > 300 else ''}")
        print(f"\nProcessing...")
        
        result = insurance_model.process_insurance_task(task_type, scenario)
        
        print(f"\nOutput:")
        print(f"{result['response']}")
        print(f"\nConfig used: {result['config_used']}")
    
    print("=" * 60)

def run_custom_demo(task_type: str, user_input: str, context: str = ""):
    """Run a custom demo with user input"""
    
    if not MODEL_LOADED:
        print("❌ Model not loaded. Please load the model first.")
        return
    
    print(f"🎯 Custom {task_type} Demo")
    print("=" * 60)
    
    if context:
        print(f"Context: {context[:200]}{'...' if len(context) > 200 else ''}")
    
    print(f"Input: {user_input}")
    print(f"\nProcessing...")
    
    result = insurance_model.process_insurance_task(task_type, user_input, context)
    
    print(f"\nOutput:")
    print(f"{result['response']}")
    print(f"\nTask: {task_type} | Config: {result['config_used']}")
    print("=" * 60)
    
    return result

def compare_generation_configs(task_type: str, user_input: str, context: str = ""):
    """Compare different generation configurations"""
    
    if not MODEL_LOADED:
        print("❌ Model not loaded. Please load the model first.")
        return
    
    print(f"🔬 Generation Config Comparison: {task_type}")
    print("=" * 80)
    
    prompt = insurance_model.format_insurance_prompt(task_type, user_input, context)
    
    for config_name, config in GENERATION_CONFIGS.items():
        print(f"\n📊 {config_name.upper()} Configuration:")
        print(f"Temperature: {config['temperature']} | Top-p: {config['top_p']} | Max tokens: {config['max_new_tokens']}")
        
        response = insurance_model.generate_response(prompt, config)
        
        print(f"Output: {response}")
        print("-" * 60)

print("✅ Demo functions ready")
print(f"Available functions:")
print(f"  run_sample_demo(task_type, sample_index=0)")
print(f"  run_custom_demo(task_type, user_input, context='')")
print(f"  compare_generation_configs(task_type, user_input, context='')")

## 6. Interactive Demos

In [ ]:
# Demo 1: Claim Classification
if MODEL_LOADED:
    print("🚗 DEMO 1: Claim Classification")
    run_sample_demo('CLAIM_CLASSIFICATION', 0)
else:
    print("⚠️ Model not loaded - skipping demo")

In [ ]:
# Demo 2: Policy Summarization
if MODEL_LOADED:
    print("📄 DEMO 2: Policy Summarization")
    run_sample_demo('POLICY_SUMMARIZATION', 0)
else:
    print("⚠️ Model not loaded - skipping demo")

In [ ]:
# Demo 3: FAQ Generation
if MODEL_LOADED:
    print("❓ DEMO 3: FAQ Generation")
    run_sample_demo('FAQ_GENERATION', 0)
else:
    print("⚠️ Model not loaded - skipping demo")

In [ ]:
# Demo 4: Compliance Check
if MODEL_LOADED:
    print("⚖️ DEMO 4: Compliance Check")
    run_sample_demo('COMPLIANCE_CHECK', 0)
else:
    print("⚠️ Model not loaded - skipping demo")

In [ ]:
# Demo 5: Contract Q&A
if MODEL_LOADED:
    print("❓ DEMO 5: Contract Q&A")
    run_sample_demo('CONTRACT_QA')
else:
    print("⚠️ Model not loaded - skipping demo")

## 7. Custom Input Testing

In [ ]:
# Custom testing area
if MODEL_LOADED:
    print("🧪 CUSTOM TESTING AREA")
    print("=" * 50)
    print("Use the functions below to test your own inputs:")
    print()
    
    # Example custom test
    custom_claim = """Patient visited emergency room after experiencing chest pain. 
    EKG and blood work performed. Diagnosis: anxiety attack, not cardiac event. 
    Patient discharged with follow-up instructions."""
    
    print("Example custom claim classification:")
    run_custom_demo('CLAIM_CLASSIFICATION', custom_claim)
    
    print("\n" + "="*50)
    print("Try your own examples by calling:")
    print("run_custom_demo('TASK_TYPE', 'your input here')")
    print("\nAvailable task types:")
    for task in TASK_TEMPLATES.keys():
        print(f"  - {task}")
        
else:
    print("⚠️ Model not loaded - custom testing not available")
    print("\nTo test custom inputs, ensure the model is loaded and use:")
    print("run_custom_demo('TASK_TYPE', 'your_input_here')")

## 8. Generation Configuration Comparison

In [ ]:
# Compare different generation configurations
if MODEL_LOADED:
    print("🔬 GENERATION CONFIGURATION COMPARISON")
    
    test_input = "Explain what comprehensive auto insurance covers and what it doesn't cover."
    
    compare_generation_configs('POLICY_SUMMARIZATION', test_input)
else:
    print("⚠️ Model not loaded - configuration comparison not available")

## 9. Interactive Widget Interface (Optional)

In [ ]:
# Interactive widget interface (if available)
if WIDGETS_AVAILABLE and MODEL_LOADED:
    
    def create_interactive_demo():
        """Create an interactive widget-based demo"""
        
        # Task selector
        task_selector = widgets.Dropdown(
            options=list(TASK_TEMPLATES.keys()),
            value=list(TASK_TEMPLATES.keys())[0],
            description='Task:',
            style={'description_width': 'initial'}
        )
        
        # Input text area
        input_text = widgets.Textarea(
            value='Enter your insurance-related text here...',
            placeholder='Type your insurance document or question here',
            description='Input:',
            layout=widgets.Layout(width='100%', height='150px')
        )
        
        # Context text area (for Q&A tasks)
        context_text = widgets.Textarea(
            value='',
            placeholder='Optional: Add context for Q&A tasks',
            description='Context:',
            layout=widgets.Layout(width='100%', height='100px')
        )
        
        # Generation config selector
        config_selector = widgets.Dropdown(
            options=list(GENERATION_CONFIGS.keys()),
            value='factual',
            description='Config:',
            style={'description_width': 'initial'}
        )
        
        # Submit button
        submit_button = widgets.Button(
            description='Generate Response',
            button_style='primary',
            layout=widgets.Layout(width='200px')
        )
        
        # Output area
        output_area = widgets.Output()
        
        def on_submit_clicked(b):
            """Handle submit button click"""
            
            with output_area:
                clear_output(wait=True)
                
                task_type = task_selector.value
                user_input = input_text.value
                context = context_text.value
                
                if not user_input or user_input.strip() == 'Enter your insurance-related text here...':
                    print("❌ Please enter some input text")
                    return
                
                print(f"🎯 Processing {task_type}...")
                print("=" * 50)
                
                try:
                    # Override default config if user selected different one
                    prompt = insurance_model.format_insurance_prompt(task_type, user_input, context)
                    generation_config = GENERATION_CONFIGS[config_selector.value]
                    response = insurance_model.generate_response(prompt, generation_config)
                    
                    print(f"✅ Response:")
                    print(response)
                    
                except Exception as e:
                    print(f"❌ Error: {e}")
        
        submit_button.on_click(on_submit_clicked)
        
        # Layout
        demo_interface = widgets.VBox([
            widgets.HTML(value="<h3>🤖 Interactive Insurance LLaMA Demo</h3>"),
            task_selector,
            input_text,
            context_text,
            config_selector,
            submit_button,
            output_area
        ])
        
        return demo_interface
    
    # Create and display the interface
    demo_widget = create_interactive_demo()
    display(demo_widget)
    
else:
    if not WIDGETS_AVAILABLE:
        print("📱 Interactive widgets not available")
        print("Use the demo functions above for testing")
    else:
        print("⚠️ Model not loaded - interactive demo not available")

## 10. Summary and Next Steps

In [ ]:
def display_summary():
    """Display summary and next steps"""
    
    print("🎉 LLaMA Insurance Model Demo Complete!")
    print("=" * 60)
    
    if MODEL_LOADED:
        print("✅ Model Status: Loaded and Ready")
        print(f"📊 Available Tasks: {len(TASK_TEMPLATES)}")
        print(f"⚙️ Generation Configs: {len(GENERATION_CONFIGS)}")
        print(f"🎯 Sample Scenarios: Available")
        
        print("\n🚀 What you can do:")
        print("1. Test different insurance scenarios with run_sample_demo()")
        print("2. Try your own inputs with run_custom_demo()")
        print("3. Compare generation settings with compare_generation_configs()")
        print("4. Use the interactive widget interface (if available)")
        
        print("\n📋 Task Types:")
        for task, desc in TASK_TEMPLATES.items():
            print(f"  • {task}: {desc['instruction'][:50]}...")
        
        print("\n💡 Tips for Best Results:")
        print("• Use 'precise' config for factual answers")
        print("• Use 'creative' config for content generation")
        print("• Use 'factual' config for summaries and explanations")
        print("• Provide clear, specific inputs")
        print("• Add context for Q&A tasks")
        
    else:
        print("❌ Model Status: Not Loaded")
        print("\n🔧 To use this demo:")
        print("1. Complete training in notebook 03_finetuning_lora.ipynb")
        print("2. Ensure model files are saved in outputs/final_model/lora_model/")
        print("3. Restart this notebook and load the model")
    
    print("\n🎯 Production Deployment:")
    print("• Consider model optimization for faster inference")
    print("• Implement proper input validation and safety checks")
    print("• Add logging and monitoring for production use")
    print("• Set up A/B testing for different configurations")
    
    print("\n📚 Project Complete!")
    print("You now have a fully functional insurance-specialized LLaMA model")
    print("=" * 60)

# Display final summary
display_summary()